# 🚀 Create Annotation Projects with Labellerr SDK

> Build complete annotation projects programmatically with custom questions and workflows

## 📚 What You'll Build

- ✅ Create projects using existing annotation templates
- ✅ Build projects with inline custom annotation questions
- ✅ Configure rotation workflows (annotation → review → client review)
- ✅ Upload datasets and manage file collections
- ✅ Understand project limitations and best practices

---

## 📋 Prerequisites

- **Google Colab** or Jupyter notebook environment
- **Labellerr API credentials** (API Key, Secret, Client ID)
- **Email address** associated with your Labellerr account
- **Dataset files** ready for upload (optional)

### 🔑 Setting up Colab Secrets
1. Click the **🔑 key icon** in the left sidebar
2. Add secrets: `LABELLERR_API_KEY`, `LABELLERR_API_SECRET`, `LABELLERR_CLIENT_ID`
3. Enable notebook access

---

## 🛠️ Installation

In [ ]:
!pip install git+https://github.com/tensormatics/SDKPython.git

---

## 🔐 Authentication Setup

In [ ]:
from labellerr.client import LabellerrClient
from labellerr.exceptions import LabellerrError

try:
    from google.colab import userdata
    api_key = userdata.get('LABELLERR_API_KEY')
    api_secret = userdata.get('LABELLERR_API_SECRET')
    client_id = userdata.get('LABELLERR_CLIENT_ID')
    print("✅ Credentials loaded from Colab Secrets")
except:
    api_key = input("Enter API Key: ")
    api_secret = input("Enter API Secret: ")
    client_id = input("Enter Client ID: ")

client = LabellerrClient(api_key, api_secret)
email = input("Enter your email: ")

print("✅ Client initialized!")
print(f"📍 Client ID: {client_id}")

---

## 📊 Quick Reference: Supported Data Types

| Data Type | Supported Formats | Use Case |
|-----------|------------------|----------|
| **image** | .jpg, .jpeg, .png, .bmp, .tiff | Object detection, segmentation, classification |
| **video** | .mp4 | Video annotation, tracking |
| **audio** | .mp3, .wav | Audio transcription, segmentation |
| **document** | .pdf | Document analysis, entity extraction |
| **text** | .txt | Text classification, NER |

---

## ⚠️ Important Limitations

> **Note:** When creating projects with file uploads:
- Maximum **2,500 files** per folder
- Total folder size should not exceed **2.5 GB**
- For larger datasets, split into multiple uploads

---

## 📂 Method 1: Create Project with Existing Template

Use this method when you have a pre-configured annotation template.

### When to use:
- Reusing annotation schemas across projects
- Standardizing annotation workflows
- Quick project creation

In [ ]:
# Project configuration with existing template
project_payload = {
    'client_id': client_id,
    'dataset_name': 'Sample_Image_Dataset',
    'dataset_description': 'A sample dataset for image annotation',
    'data_type': 'image',  # Options: image, video, audio, document, text
    'created_by': email,
    'project_name': 'My First Annotation Project',
    'annotation_template_id': 'your_template_id_here',  # Replace with actual template ID
    'rotation_config': {
        'annotation_rotation_count': 1,  # How many times each file is annotated
        'review_rotation_count': 1,      # How many times annotations are reviewed
        'client_review_rotation_count': 1  # How many times client reviews
    },
    'autolabel': False,  # Enable AI-assisted labeling
    # 'folder_to_upload': '/path/to/your/images'  # Optional: upload files
}

try:
    result = client.initiate_create_project(project_payload)
    project_id = result['project_id']['response']['project_id']
    print(f"✅ Project created successfully!")
    print(f"📁 Project ID: {project_id}")
    print(f"📛 Project Name: {project_payload['project_name']}")
except LabellerrError as e:
    print(f"❌ Project creation failed: {str(e)}")

---

## 📝 Method 2: Create Project with Inline Annotation Questions

Create projects with custom annotation questions defined inline.

### When to use:
- Creating unique annotation schemas
- Prototyping new annotation workflows
- Full control over annotation structure

In [ ]:
import uuid

# Define annotation questions
annotation_guide = [
    {
        "question_number": 1,
        "question": "Vehicle Type",
        "question_id": str(uuid.uuid4()),
        "option_type": "BoundingBox",  # Draw bounding boxes
        "required": True,
        "options": [
            {"option_name": "Car"},
            {"option_name": "Truck"},
            {"option_name": "Motorcycle"}
        ],
        "question_metadata": []
    },
    {
        "question_number": 2,
        "question": "Is the vehicle moving?",
        "question_id": str(uuid.uuid4()),
        "option_type": "boolean",  # Yes/No question
        "required": True,
        "options": []
    },
    {
        "question_number": 3,
        "question": "Weather Condition",
        "question_id": str(uuid.uuid4()),
        "option_type": "dropdown",  # Dropdown selection
        "required": False,
        "options": [
            {"option_id": str(uuid.uuid4()), "option_name": "Sunny"},
            {"option_id": str(uuid.uuid4()), "option_name": "Rainy"},
            {"option_id": str(uuid.uuid4()), "option_name": "Cloudy"},
            {"option_id": str(uuid.uuid4()), "option_name": "Foggy"}
        ]
    }
]

print("✅ Annotation guide created with 3 questions:")
for q in annotation_guide:
    print(f"   {q['question_number']}. {q['question']} ({q['option_type']})")

In [ ]:
# Create project with inline annotation guide
project_payload = {
    'client_id': client_id,
    'dataset_name': 'Vehicle_Detection_Dataset',
    'dataset_description': 'Dataset for vehicle detection and classification',
    'data_type': 'image',
    'created_by': email,
    'project_name': 'Vehicle Detection Project',
    'annotation_guide': annotation_guide,  # Use inline questions
    'rotation_config': {
        'annotation_rotation_count': 1,
        'review_rotation_count': 1,
        'client_review_rotation_count': 1
    },
    'autolabel': False
}

try:
    result = client.initiate_create_project(project_payload)
    project_id = result['project_id']['response']['project_id']
    print(f"✅ Project created successfully with custom questions!")
    print(f"📁 Project ID: {project_id}")
    print(f"📛 Project Name: {project_payload['project_name']}")
    print(f"❓ Questions: {len(annotation_guide)}")
except LabellerrError as e:
    print(f"❌ Project creation failed: {str(e)}")

---

## 🔄 Understanding Rotation Configs

Rotation configs control the annotation workflow:

```
File → Annotation (x1) → Review (x1) → Client Review (x1) → Accepted
```

### Configuration Options:

| Parameter | Description | Typical Value |
|-----------|-------------|---------------|
| `annotation_rotation_count` | How many annotators label each file | 1-3 |
| `review_rotation_count` | How many reviewers check each annotation | 1-2 |
| `client_review_rotation_count` | How many client reviewers approve | 1 |

### Example Scenarios:

**High Quality (Multiple Annotators):**
```python
'rotation_config': {
    'annotation_rotation_count': 3,  # 3 different annotators
    'review_rotation_count': 2,      # 2 reviewers
    'client_review_rotation_count': 1
}
```

**Fast Turnaround (Single Pass):**
```python
'rotation_config': {
    'annotation_rotation_count': 1,
    'review_rotation_count': 1,
    'client_review_rotation_count': 1
}
```

---

## 📤 File Upload Options

### Option 1: Upload Folder
```python
'folder_to_upload': '/content/my_images'
```

### Option 2: Upload Specific Files
```python
'files_to_upload': [
    '/content/image1.jpg',
    '/content/image2.jpg',
    '/content/image3.jpg'
]
```

### Best Practices:
- For small datasets (< 100 files): Use either method
- For large datasets: Split into batches of 2,000 files
- Use descriptive file names for easier management

---

## 🎯 Complete End-to-End Example

Let's create a complete project from scratch:

In [ ]:
# Step 1: Define annotation schema
annotation_schema = [
    {
        "question_number": 1,
        "question": "Object Detection",
        "question_id": str(uuid.uuid4()),
        "option_type": "polygon",
        "required": True,
        "options": [{"option_name": "#FF0000"}],
        "question_metadata": []
    },
    {
        "question_number": 2,
        "question": "Quality Check",
        "question_id": str(uuid.uuid4()),
        "option_type": "radio",
        "required": True,
        "options": [
            {"option_id": str(uuid.uuid4()), "option_name": "Good Quality"},
            {"option_id": str(uuid.uuid4()), "option_name": "Poor Quality"},
            {"option_id": str(uuid.uuid4()), "option_name": "Needs Review"}
        ]
    }
]

# Step 2: Create project payload
complete_project = {
    'client_id': client_id,
    'dataset_name': 'Complete_Example_Dataset',
    'dataset_description': 'End-to-end project creation example',
    'data_type': 'image',
    'created_by': email,
    'project_name': 'Complete Example Project',
    'annotation_guide': annotation_schema,
    'rotation_config': {
        'annotation_rotation_count': 1,
        'review_rotation_count': 1,
        'client_review_rotation_count': 1
    },
    'autolabel': False
}

# Step 3: Create project
try:
    print("📝 Creating project...")
    result = client.initiate_create_project(complete_project)
    
    project_id = result['project_id']['response']['project_id']
    
    print("\n" + "="*70)
    print("✅ PROJECT CREATED SUCCESSFULLY!")
    print("="*70)
    print(f"📁 Project ID: {project_id}")
    print(f"📛 Name: {complete_project['project_name']}")
    print(f"📊 Data Type: {complete_project['data_type']}")
    print(f"❓ Questions: {len(annotation_schema)}")
    print(f"🔄 Workflow: Annotate → Review → Client Review")
    print("="*70)
    
    # Store for later use
    created_project_id = project_id
    
except LabellerrError as e:
    print(f"❌ Error: {str(e)}")

---

## 📖 Project Payload Reference

| Field | Type | Required | Description |
|-------|------|----------|-------------|
| `client_id` | String | Yes | Your client identifier |
| `dataset_name` | String | Yes | Name for the dataset |
| `dataset_description` | String | Yes | Dataset description |
| `data_type` | String | Yes | image, video, audio, document, or text |
| `created_by` | String | Yes | Your email address |
| `project_name` | String | Yes | Name for the project |
| `annotation_template_id` | String | No* | ID of existing template |
| `annotation_guide` | Array | No* | Inline annotation questions |
| `rotation_config` | Object | Yes | Workflow configuration |
| `autolabel` | Boolean | Yes | Enable AI-assisted labeling |
| `folder_to_upload` | String | No | Path to folder to upload |
| `files_to_upload` | Array | No | List of file paths |

*Either `annotation_template_id` OR `annotation_guide` is required

---

## 🔧 Troubleshooting Common Issues

### Issue 1: "annotation_template_id or annotation_guide required"
**Solution:** Provide either a template ID or inline annotation guide.

### Issue 2: "File size exceeds limit"
**Solution:** Split your dataset into smaller batches (< 2.5 GB each).

### Issue 3: "Invalid data_type"
**Solution:** Use one of: `image`, `video`, `audio`, `document`, `text`.

### Issue 4: "Authentication failed"
**Solution:** Verify your API credentials in Colab Secrets.

---

## 🎯 Next Steps

Congratulations! You've mastered creating annotation projects! 🎉

### Recommended Next Steps:

1. **Master Annotation Questions** - Learn to create complex annotation schemas
   - 📓 [03_annotation_questions.ipynb](./03_annotation_questions.ipynb)

2. **Upload Pre-Annotations** - Speed up your workflow
   - 📓 [04_upload_preannotations.ipynb](./04_upload_preannotations.ipynb)

3. **Retrieve Projects** - Manage existing projects
   - 📓 [01_retrieve_projects_datasets.ipynb](./01_retrieve_projects_datasets.ipynb)

### Additional Resources:

- 📖 [Labellerr Documentation](https://docs.labellerr.com)
- 🌐 [SDK GitHub](https://github.com/tensormatics/SDKPython)
- 📧 **Support**: support@tensormatics.com

---

### 💡 Pro Tips:

- Start with small test projects before scaling
- Use descriptive project and dataset names
- Configure rotation counts based on quality requirements
- Save project IDs for later reference

---

**Happy Annotating! 🚀**